### Python-SQL Automated ETL and Export Pipeline

#### *Import libraries.*

• pyodbc for connecting to database.<br>• pandas to handle dataframe.<br>• csv to handle data export from database to .csv file.

In [ ]:
import pyodbc
from tabulate import tabulate
import pandas as pd
import csv

#### *Extract data from csv*

Using pandas, reading csv file and extracting data to a dataframe. Checking the first 3 rows of the dataframe.

In [ ]:
df=pd.read_csv("employees$.csv")
df.head(3)

#### *Transform Data in Dataframe*

Now that our dataframe is ready, we can clean and transform the data for further process. So we're checking some info of our dataframe.

In [ ]:
df.info()

We can see there are no null values. But something is suspecious here. So we will check the values of major columns.

In [ ]:
df["COMMISSION PCT"].unique()

As we can see, this column contains only '-' and no actual value that specifies the column name. This column is useless for now, so will drop it.<br>If required we'll create it when needed.

In [ ]:
df=df.drop("COMMISSION PCT", axis=1)
df.head(3)

Done. we'll check the same for some other columns too untill we find any non-null dummy value.

In [ ]:
df["EMPLOYEE ID"].unique()

In [ ]:
df["DEPARTMENT ID"].unique()

In [ ]:
df["MANAGER ID"].unique()

Here we can see there is as unusual data that doesn't match with the data present in the column. We need to handle this.<br>So lets go down to the logic here. The manager id is missing for a department. What we can do is,<br>• Check the dept id of the missing manager id.<br>• Find the same dept id where manager id is mentioned.<br>• Fill the missing manager id with the id fetched by comparing dept id.

In [ ]:
df["MANAGER ID"]=df["MANAGER ID"].astype(str)
mis=df[df["MANAGER ID"]==" - "].index

for i in mis:
    dept=df.loc[i,"DEPARTMENT ID"]
    mng=df.loc[(df["DEPARTMENT ID"]==dept) & (df["MANAGER ID"]!=" - "),"MANAGER ID"]
    if not mng.empty:
        df.loc[i,"MANAGER ID"]=mng.values[0]

df["MANAGER ID"]=df["MANAGER ID"].astype("int64")
df["MANAGER ID"].unique()

Now we'll clean the file name for our table name

In [ ]:
f_nm="employees$"
cl_tbl_nm=f_nm.replace("$","")
cl_tbl_nm

Same we'll do to the column names for our table

In [ ]:
df.columns=[x.lower().replace(" ","_") for x in df.columns]
df.columns

Now we'll check the data type of our columns in dataframe as we need to replace it with data type for our SQL server while executing our query

In [ ]:
df.dtypes

So here, we're providing alternate data types for our SQL query, joining it with column names and making it ready to copy for our query.

In [ ]:
typcnvr={
    'int64':'int',
    'object':'varchar'
}
cl_nm=', '.join("{} {}".format(n, d) for (n ,d) in zip(df.columns, df.dtypes.replace(typcnvr)))
cl_nm

So our data is ready to be loaded to database.<br>We'll first export our dataframe to csv file so that we can read and pass it to our query.

In [ ]:
df.to_csv('employee.csv', header=df.columns, index=False,encoding='utf-8')

#### *Connect py script to database.*

Here I'm connecting to SQL Server using windows authentication. Establishing a cursor that'll pass our queries to server. Using try and except method to fetch error(if any) and display.

In [ ]:
try:
    conn=pyodbc.connect(
        'Driver={Driver Name};'
        'Server=Server Name;'
        'Database=Database Name;'
        'Trusted_Connection=Yes;'
    )
    print("Server Connected Successfully! opened database")
    cursor=conn.cursor()
    print("Cursor ready!")
except pyodbc.Error as e:
    print("Error in establishing connection: ",e)

Using cursor, we'll pass a query to create a table. Here we can copy the standardized table name from cl_tbl_nm and column name from cl_nm. After which we'll commit the query to our connected SQL database.

In [ ]:
try:
    cursor.execute("""
        CREATE TABLE employees(
            employee_id int,
            first_name varchar(20),
            last_name varchar(20),
            email varchar(30),
            phone_number varchar(15),
            hire_date varchar(10),
            job_id varchar(10),
            salary int,
            manager_id int,
            department_id int
        );
    """)
    conn.commit()
    print("Table created successfully!")
except pyodbc.Error as e:
    print("Error in creating table: ", e)

#### *Load data to Database.*

Now that we created our desired table, we can load our data to database. The data we transformed and saved in csv, we'll<br>• Open that file<br>• Read each line, split as per delimiter ',' and create a list of those data<br>• Load each item of the list as per columns via insert query

In [ ]:
try:
    l=0
    with open('employee.csv') as d_file:
        next(d_file)    #skip the first row, containinig headers.
        for line in d_file:
            values=line.strip().split(',')
            cursor.execute("INSERT INTO employees VALUES(?,?,?,?,?,?,?,?,?,?);",values)
            l+=1    #counting no. of rows inserted
    conn.commit()
    print(f"Data loaded to db with {l} rows!")
except pyodbc.Error as e:
    print("Error in copying data from csv to db: ",e)

After inserting data, let's look into our table. Tabulate is used to display the fetched data in tabular format.

In [ ]:
try:
    cursor.execute("SELECT TOP 5 * FROM employees;")
    print(tabulate(cursor.fetchall(), headers=[c[0] for c in cursor.description], tablefmt='grid'))
except pyodbc.Error as e:
    print("Error: ", e)

Let us add some more data in the database with query passed to database via cursor.

In [ ]:
try:
    cursor.execute("""INSERT INTO employees VALUES
                    (141, 'Susan', 'Mavrick', 'SMAVRICK', '515.123.7777', '10-Oct-06', 'MK_REP', 6200, 201, 20),
                    (142, 'Chris', 'Taylor', 'CTAYLOR', '650.507.8888', '01-Mar-07', 'SH_CLERK', 2700, 124, 50),
                    (143, 'Nancy', 'Jones', 'NJONES', '515.123.9999', '12-Dec-02', 'AD_ASST', 4500, 101, 10),
                    (144, 'Matthew', 'Smith', 'MSMITH', '603.123.1234', '22-May-09', 'IT_PROG', 9200, 102, 60),
                    (145, 'Emily', 'Brown', 'EBROWN', '515.123.4321', '15-Jul-10', 'IT_PROG', 8700, 102, 60),
                    (146, 'Robert', 'White', 'RWHITE', '650.507.5678', '19-Jan-12', 'ST_CLERK', 3100, 203, 50),
                    (147, 'Sophia', 'Green', 'SGREEN', '603.123.6543', '23-Jun-08', 'ST_CLERK', 2800, 203, 50),
                    (148, 'William', 'Black', 'WBLACK', '515.123.8888', '30-Apr-11', 'HR_REP', 8000, 200, 40),
                    (149, 'Linda', 'Morris', 'LMORRIS', '650.507.2468', '10-Nov-13', 'HR_REP', 7500, 200, 40),
                    (150, 'James', 'Wilson', 'JWILSON', '603.123.7890', '27-Aug-14', 'SH_CLERK', 2900, 124, 50);                   
                   """)
    print("Data inserted!")
except pyodbc.Error as e:
    print("Error: ", e)

#### *Exporting Data from database.*

##### First method is exporting data from database directly to csv.

Fetch required data with select query and store in a variable(rows). Fetch columns and store in a variable(columns). Create and open a csv file and create a writer object to write data to csv file. Load column name and rows accordingly into the file.

In [ ]:
try:
    cursor.execute("SELECT * FROM employees")
    rows=cursor.fetchall()
    columns=[c[0] for c in cursor.description]
    with open('emp_dr_csv.csv',mode='w+',newline='',encoding='utf-8') as file:
        data=csv.writer(file)
        data.writerow(columns)
        data.writerows(rows)
    print("Data exported from database to csv file!")

except pyodbc.Error as e:
    print("Error: ",e)

##### Second method is by exporting data from database to dataframe and then to required file type.

First we'll read the output of the SQL query and load it to a dataframe using pandas.<br>This method let us perform further transformations and analysis before exporting to file.

In [ ]:
try:
    df=pd.read_sql("SELECT * FROM employees", conn)
    print("Data loaded from database to dataframe!")
except pyodbc.Error as e:
    print("Error: ",e)

When the data is loaded to dataframe, we can perform various analysis as per requirement.

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

Let's perform some simple EDA on our data.<br>Here I want to know about the Name, Job id and salary of the employee who gets the maximum and minimum salaries in the team.

In [ ]:
l_mx=df.loc[df['salary']==df['salary'].max()]
print(f"{l_mx['first_name'].values[0].upper()} {l_mx['last_name'].values[0].upper()} with job id '{l_mx['job_id'].values[0]}' gets the highest salary ${l_mx['salary'].values[0]}")
l_mn=df.loc[df['salary']==df['salary'].min()]
print(f"{l_mn['first_name'].values[0].upper()} {l_mn['last_name'].values[0].upper()} with job id '{l_mn['job_id'].values[0]}' gets the lowest salary ${l_mn['salary'].values[0]}")

Likewise we can perform further EDA, data manipulation, visualizations etc. before exporting the data.

After which, we are ready to export the data, that we extracted from database, from dataframe to to desired file type.<br>So first we'll export to csv format with proper encoding. Provide desired file name, a csv file will be created in the directory containing extracted data.

In [ ]:
df.to_csv('emp_df_csv.csv',index=False,encoding='utf-8')
print("Data exported from dataframe to csv file!")

Secondly, we can export the data to excel file from dataframe. Provide desired file name, a xlsx file will be created in the directory containing extracted data.

In [ ]:
df.to_excel('emp_df_xlsx.xlsx',index=False,sheet_name='Sheet1')
print("Data exported from dataframe to xlsx file!")

#### *Close connection between py script and database.*

Close the connection any time when task is completed and there are no more requirements of the connection to database. If the connection remains open and any query is passed to the database by mistake, it'll hamper the data in database.<br>Closing connection immediate to end of task with database is suggested

In [ ]:
conn.close()